# Next.js Multi-Zone App with Shared Components

This notebook provides a comprehensive guide for setting up and using shared components across a multi-zone Next.js application. It explains the different approaches available, configuration steps, and best practices.

## Approaches for Sharing Components

There are three main approaches for sharing components between Next.js apps:

1. **npm link (Development)** - Good for local development
2. **Workspace Dependencies** - Best for monorepo setups
3. **Published Package** - Best for production/deployment

Let's explore each approach.

## Approach 1: npm link (Development)

The `npm link` approach creates a symbolic link between your shared component package and the apps that consume it. This is great for development because changes to shared components are immediately available to consumer apps.

### Steps for setting up npm link:

In [ ]:
// Step 1: In your shared components directory
// Create a package.json file if not exists
/*
{
  "name": "shared-components",
  "version": "1.0.0",
  "main": "src/index.js",
  "private": true,
  "scripts": {
    "dev": "echo \"No dev server for component library\" && exit 0",
    "build": "echo \"No build needed for direct import\" && exit 0"
  },
  "dependencies": {
    "react": "^18.2.0",
    "react-dom": "^18.2.0",
    "styled-components": "^6.0.0"
  },
  "peerDependencies": {
    "next": "^13.0.0 || ^14.0.0 || ^15.0.0",
    "react": "^17.0.0 || ^18.0.0 || ^19.0.0",
    "react-dom": "^17.0.0 || ^18.0.0 || ^19.0.0"
  }
}
*/

// Step 2: Register the package with npm link
// Run this command in the shared-components directory
// $ npm link

// Step 3: Link to the package in each app
// Run this command in each app directory (host-app, app1, etc.)
// $ npm link shared-components

### Typical File Structure for Shared Components:

```
shared-components/
├── package.json
├── src/
│   ├── index.js         # Main entry point that exports all components
│   └── Components/
│       ├── Button.js
│       ├── Navbar.js
│       └── DataTable.js
```

### Sample Component Implementation:

In [ ]:
// Button.js
"use client"; // This is important for Next.js client components

import React from 'react';
import styled from 'styled-components';

const StyledButton = styled.button`
  padding: 10px 15px;
  border-radius: 4px;
  font-weight: 600;
  cursor: pointer;
  transition: all 0.2s ease;
  background-color: ${(props) => props.variant === 'primary' ? '#0070f3' : '#ffffff'};
  color: ${(props) => props.variant === 'primary' ? '#ffffff' : '#0070f3'};
  border: ${(props) => props.variant === 'primary' ? 'none' : '1px solid #0070f3'};

  &:hover {
    opacity: 0.9;
    transform: translateY(-1px);
  }
`;

const Button = ({ children, variant = 'primary', onClick, ...props }) => {
  return (
    <StyledButton 
      variant={variant} 
      onClick={onClick}
      {...props}
    >
      {children}
    </StyledButton>
  );
};

export default Button;

In [ ]:
// index.js - Main entry point for shared components
import Button from './Components/Button';
import Navbar from './Components/Navbar';
import DataTable from './Components/DataTable';

export {
  Button,
  Navbar,
  DataTable
};

## Approach 2: Workspace Dependencies (Monorepo)

For a more robust setup, especially in larger projects, using a monorepo approach with workspace dependencies is recommended. This can be done using npm workspaces or a tool like Turborepo.

### Setting up npm workspaces:

In [ ]:
// Root package.json
/*
{
  "name": "my-multizone-app",
  "version": "1.0.0",
  "private": true,
  "workspaces": [
    "host-app",
    "app1",
    "app2",
    "shared-components"
  ],
  "scripts": {
    "dev": "./start-zones.sh",
    "build": "npm run build --workspaces"
  }
}
*/

// In each app's package.json
/*
{
  "name": "host-app",
  "dependencies": {
    "shared-components": "*",  // Workspace dependency
    "next": "^15.0.0",
    // other dependencies...
  }
}
*/

## Next.js Configuration for Shared Components

Each Next.js app needs to be configured to properly transpile the shared components package. This is done in the `next.config.js` (or `.mjs`) file:

In [ ]:
// next.config.mjs in each app
/**
 * @type {import('next').NextConfig}
 */
const nextConfig = {
  // This is crucial - it tells Next.js to transpile the shared-components package
  transpilePackages: ['shared-components'],
  
  // Other Next.js configuration...
  reactStrictMode: true,
  
  // For multi-zone apps
  basePath: '/app1', // For sub-apps
  
  // Rewrites for multi-zone
  async rewrites() {
    return [
      {
        source: '/app2/:path*',
        destination: 'http://localhost:3002/:path*',
      },
    ];
  }
};

export default nextConfig;

## Using Shared Components in Next.js Apps

Now that we've set up our shared components, let's look at how to use them in our Next.js applications:

### Example 1: Using a shared Button component

In [ ]:
// app1/src/app/page.js
"use client"; // Important when using client components

import { Button } from 'shared-components';

export default function Home() {
  return (
    <div>
      <h1>Welcome to App 1</h1>
      <Button onClick={() => alert('Button clicked!')}>
        Click Me
      </Button>
      <Button variant="secondary">
        Secondary Button
      </Button>
    </div>
  );
}

### Example 2: Using a shared component with state lifting

In [ ]:
// app2/src/app/page.js
"use client";

import { useState } from 'react';
import { SearchInput, DataTable } from 'shared-components';

export default function SearchPage() {
  const [searchResults, setSearchResults] = useState([]);
  
  const columns = [
    { key: 'id', title: 'ID' },
    { key: 'name', title: 'Name' },
    { key: 'email', title: 'Email' }
  ];
  
  // This function will be passed to the SearchInput component
  // Example of state lifting
  const handleSearch = (term) => {
    // In a real app, you'd fetch data from an API
    const mockResults = [
      { id: 1, name: 'John Doe', email: 'john@example.com' },
      { id: 2, name: 'Jane Smith', email: 'jane@example.com' }
    ].filter(user => 
      user.name.toLowerCase().includes(term.toLowerCase())
    );
    
    setSearchResults(mockResults);
  };
  
  return (
    <div>
      <h1>User Search</h1>
      
      <SearchInput 
        onSearch={handleSearch}
        placeholder="Search users..."
      />
      
      <DataTable 
        columns={columns}
        data={searchResults}
      />
    </div>
  );
}

## Best Practices for Shared Components

1. **Always add "use client" directive** to components using React hooks, browser APIs, or client-side functionality
2. **Keep dependencies consistent** across all apps and shared components
3. **Design components to be reusable** with flexible props
4. **Document your components** with clear examples
5. **Consider a component storybook** for visual documentation
6. **Use TypeScript** for better type safety and IDE support

## Troubleshooting Common Issues

### 1. Component not found or "Module not found" errors
- Check that the component is properly exported from index.js
- Verify that transpilePackages is correctly configured
- Make sure npm link was successful

### 2. "use client" errors
- Add "use client" directive to any component that uses client features
- Check if the component is being used in a server component

### 3. Styling issues
- Ensure styled-components is installed in all packages
- Check for CSS conflicts between apps

### 4. Changes not reflected
- In npm link setup, sometimes changes aren't reflected immediately
- Try running `npm link shared-components` again in the consuming app

## Full Workflow Example

Here's a complete workflow for developing with shared components:

1. Make changes to shared components
2. If using npm link, the changes are immediately available
3. If using workspaces, may need to rebuild
4. Restart Next.js development servers if needed
5. Test your changes across all apps

This workflow ensures that your shared components can be developed and tested efficiently across your multi-zone Next.js application.